# Student Loan Risk with Deep Learning

In [ ]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [ ]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [ ]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [ ]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [ ]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"]

# Display a sample of y
y.head(20)

0     0
1     0
2     0
3     1
4     0
5     0
6     0
7     1
8     1
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    1
17    0
18    0
19    1
Name: credit_ranking, dtype: int64

In [ ]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop(columns="credit_ranking")

# Review the features DataFrame
X.head(20)

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
5,7.4,0.660,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4
6,7.9,0.600,0.06,1.6,0.069,15.0,59.0,0.9964,3.30,0.46,9.4
7,7.3,0.650,0.00,1.2,0.065,15.0,21.0,0.9946,3.39,0.47,10.0
8,7.8,0.580,0.02,2.0,0.073,9.0,18.0,0.9968,3.36,0.57,9.5
9,7.5,0.500,0.36,6.1,0.071,17.0,102.0,0.9978,3.35,0.80,10.5


### Step 3: Split the features and target sets into training and testing datasets.


In [ ]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [ ]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train_scaled

array([[-0.56794375, -1.18088385, -0.19241296, ...,  0.73249574,
        -0.22553894, -0.31915735],
       [-0.00940983,  0.80065564, -0.04123135, ..., -0.03016065,
        -0.75844691, -1.15783431],
       [ 2.33643265, -0.84597577,  1.67216021, ..., -1.36480934,
         0.42579302,  0.05358797],
       ...,
       [-0.9589175 ,  1.07974571, -0.89792713, ...,  0.22405814,
        -0.9952949 ,  0.33314696],
       [-0.62379715,  0.54947458, -1.35147195, ...,  0.35116754,
        -0.46238693, -1.34420697],
       [ 0.43741731, -0.73433974,  1.16822152, ..., -0.66570765,
         0.18894504,  1.91731456]])

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [ ]:
# Define the the number of inputs (features) to the model
input_nodes = len(X.columns)

# Review the number of features
input_nodes

11

In [ ]:
# Define the number of hidden nodes for the first hidden layer
nodes_layer_1 = 6

# Define the number of hidden nodes for the second hidden layer
nodes_layer_2 = 3

# Define the number of neurons in the output layer
op_neurons = 1

In [ ]:
# Create the Sequential model instance
nn_model = tf.keras.models.Sequential()

# Add the first hidden layer
nn_model.add(tf.keras.layers.Dense(units=nodes_layer_1, activation="relu", input_dim=input_nodes))

# Add the second hidden layer
nn_model.add(tf.keras.layers.Dense(units=nodes_layer_2, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
nn_model.add(tf.keras.layers.Dense(units=op_neurons, activation="sigmoid"))

In [ ]:
# Display the Sequential model summary
nn_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 6)                 72        
                                                                 
 dense_4 (Dense)             (None, 3)                 21        
                                                                 
 dense_5 (Dense)             (None, 1)                 4         
                                                                 
Total params: 97 (388.00 Byte)
Trainable params: 97 (388.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [ ]:
# Compile the Sequential model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Fit the model using 50 epochs and the training data
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
38/38 [==============================] - 2s 4ms/step - loss: 0.6919 - accuracy: 0.5346
Epoch 2/50
38/38 [==============================] - 0s 3ms/step - loss: 0.6819 - accuracy: 0.6047
Epoch 3/50
38/38 [==============================] - 0s 3ms/step - loss: 0.6735 - accuracy: 0.6555
Epoch 4/50
38/38 [==============================] - 0s 4ms/step - loss: 0.6647 - accuracy: 0.6664
Epoch 5/50
38/38 [==============================] - 0s 3ms/step - loss: 0.6553 - accuracy: 0.6631
Epoch 6/50
38/38 [==============================] - 0s 3ms/step - loss: 0.6452 - accuracy: 0.6722
Epoch 7/50
38/38 [==============================] - 0s 3ms/step - loss: 0.6343 - accuracy: 0.6856
Epoch 8/50
38/38 [==============================] - 0s 3ms/step - loss: 0.6241 - accuracy: 0.6989
Epoch 9/50
38/38 [==============================] - 0s 3ms/step - loss: 0.6148 - accuracy: 0.7048
Epoch 10/50
38/38 [==============================] - 0s 2ms/step - loss: 0.6060 - accuracy: 0.7164
Epoch 11/50
38/38 [

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [44]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test, verbose=2)

# Display the model loss and accuracy results
print(f"The loss is: {model_loss}, and the accuracy is: {model_accuracy}")

13/13 - 0s - loss: 0.5296 - accuracy: 0.7550 - 45ms/epoch - 3ms/step
The loss is: 0.5296448469161987, and the accuracy is: 0.7549999952316284


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [45]:
# Set the model's file path
file_path = Path("student_loans.keras")

# Export your model to a keras file
nn_model.save(file_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [46]:
# Set the model's file path
file_path = Path("student_loans.keras")

# Load the model to a new object
nn_import = tf.keras.models.load_model(file_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [47]:
# Make predictions with the test data
predict = nn_import.predict(X_test_scaled, verbose=2)

# Display a sample of the predictions
predict

13/13 - 0s - 78ms/epoch - 6ms/step


array([[0.38683796],
       [0.3132563 ],
       [0.9382243 ],
       [0.7718269 ],
       [0.98265284],
       [0.793868  ],
       [0.96263534],
       [0.30321446],
       [0.3860213 ],
       [0.43369114],
       [0.9426971 ],
       [0.30321446],
       [0.61936116],
       [0.93078464],
       [0.51326185],
       [0.30443975],
       [0.9477073 ],
       [0.3090998 ],
       [0.6751079 ],
       [0.3104379 ],
       [0.50392   ],
       [0.89736855],
       [0.30321446],
       [0.8922378 ],
       [0.30321446],
       [0.96438825],
       [0.89280134],
       [0.5746607 ],
       [0.30321446],
       [0.38362744],
       [0.76600635],
       [0.9634046 ],
       [0.30321446],
       [0.95530766],
       [0.30321446],
       [0.7479273 ],
       [0.30321446],
       [0.32379898],
       [0.93386215],
       [0.30321446],
       [0.9447575 ],
       [0.30321446],
       [0.30321446],
       [0.9755435 ],
       [0.30321446],
       [0.78631264],
       [0.30321446],
       [0.322

In [48]:
# Save the predictions to a DataFrame and round the predictions to binary results
predict_df = pd.DataFrame(columns=["predict"], data=predict)
predict_df["predict"] = round(predict_df["predict"],0)
predict_df

,predict
0,0.0
1,0.0
2,1.0
3,1.0
4,1.0
...,...
395,1.0
396,0.0
397,1.0
398,0.0


### Step 4: Display a classification report with the y test data and predictions

In [50]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, predict_df["predict"].values))

              precision    recall  f1-score   support

           0       0.72      0.78      0.75       188
           1       0.79      0.73      0.76       212

    accuracy                           0.76       400
   macro avg       0.76      0.76      0.75       400
weighted avg       0.76      0.76      0.76       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.
--- While an accuracy score of .76 is pretty good for some other modules, when it comes to student loan repayment more data is needed. Assuming that Approvals are a 1 and Denials are 0 then more data on what constitutes a loan approval is needed, so that we avoid having false positives from the data, especially if the original sample meant someone got denied 0, which later got approved for some reason.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.
--- I think the best option is content filtering, as that would give us additional filters based on the individual characteristics of the data, especially if it’s demographic and financial info (i.e. who is the person and what does their credit history look like). Another piece of info to have is the loan terms and if that aligns with the payment successes of the person based on their credit history.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.
--- The main concern I have is data privacy and security as I recently found out through my credit report that my student loan provider was hacked, and now all my personal, credit and loan history is somewhere outside of the company I was working with. I think they have enough information to be able to apply to more loans and that is a concern, however, if I was the bank, that is of little concern to me and I would instead be worried about collecting information on the person and the country’s changing economic conditions (which of course are super hard to predict) but if there is high unemployment, it may affect some of the people who have my loans, and if they don’t have a job, then they cannot pay their loan to the bank.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**


**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**


**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

